In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE, RandomOverSampler

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
train_data = pd.read_csv('NSL-KDD-Dataset/KDDTrain+.csv')
test_data = pd.read_csv('NSL-KDD-Dataset/KDDTest+.csv')
#train_data[train_data['service']!='-']
cols_to_transform = ['protocol_type', 'services', 'flag']
#train_data = pd.get_dummies(data = train_data, columns = cols_to_transform)
#test_data = pd.get_dummies(data = test_data, columns = cols_to_transform)
#train_data[['label','attack_cat']][train_data['attack_cat']!='Dos'].head(10000)
#obj_df = train_data.select_dtypes(include=['object']).copy()
for col in cols_to_transform:
    train_data[col]= train_data[col].astype('category').cat.codes
    test_data[col]= test_data[col].astype('category').cat.codes
train_data_ = train_data.drop(['num_outbound_cmds','class_label'], axis=1)
test_data_ = test_data.drop(['num_outbound_cmds','class_label'], axis=1)
train_data_ = train_data_.as_matrix(columns = None)
test_data_ = test_data_.as_matrix(columns = None)
test_data_.shape


(22545, 41)

In [3]:
x_train= train_data_[:,:-1]
y_train = train_data_[:,-1]
x_test = test_data_[:,:-1]
y_test = test_data_[:,-1]
x_min = []
x_max = []
for i in range(x_train.shape[1]):
    x_min.append(np.min(x_train[:,i]))
    x_max.append(np.max(x_train[:,i]))
x_max = np.array(x_max)
x_min = np.array(x_min)
x_train = (x_train - x_min)/(x_max-x_min)
x_test = (x_test - x_min)/(x_max-x_min)
x_test = x_test[:-1]

In [4]:
# Number of Epochs
dim_size = 18
epochs = 20
num_seq = 100
# Batch Size
batch_size =200
# RNN Size
rnn_size =50
# Learning Rate
learning_rate = 0.00008
num_layers = 1
# Show stats for every n number of batches
show_every_n_batches = 100
keep_prob = 1
relu_size = 50
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

In [5]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components = dim_size)
x_reduced = svd.fit_transform(x_train)
x_reduced[:10]

array([[ 1.32236566e+00, -2.28826003e-01,  9.68252263e-02,
         4.86907317e-01, -3.37092847e-01,  3.10851823e-01,
        -2.20936845e-01,  1.84154175e-01, -3.05602369e-01,
         1.16690320e-01,  8.43966141e-02, -1.07548648e-01,
        -1.61493224e-01,  4.74326274e-02,  1.16950327e-01,
        -1.34933275e-01, -2.40885055e-02, -9.48888489e-02],
       [ 1.35046069e+00,  2.59793799e-01,  4.35092859e-01,
         7.80733812e-01, -6.45491350e-01,  7.52956059e-01,
        -9.14937032e-02,  1.68794485e-02,  4.23706159e-01,
        -3.87287405e-01, -1.98567432e-01,  4.07497663e-01,
         2.55590568e-01,  3.77393376e-02, -3.81232140e-02,
         9.42956877e-03,  2.12495512e-01, -1.80928064e-02],
       [ 1.60048583e+00,  1.84573252e+00, -2.73237501e-01,
        -5.69528671e-02,  1.09015601e-01, -1.97459521e-02,
        -1.44628636e-03,  5.26880667e-02,  5.31515125e-02,
         6.27603752e-03, -1.58721403e-01,  1.62501842e-03,
        -9.93925572e-03,  1.05593398e-02,  9.74464693e

In [6]:
x_test_reduced = svd.transform(x_test)

In [7]:
x_train.shape

(125973, 40)

In [8]:
def resample(arr,y, num_seq):
    x_resampled ,y_resampled = SMOTE().fit_sample(arr, y)
    return x_resampled,y_resampled


In [9]:
x_resampled = []
y_resampled = []
for i in range(y_train.size//25000):
    x_resampledi, y_resampledi = resample(x_reduced[i*25000:25000*(i+1)], y_train[i*25000:25000*(i+1)],num_seq)
    x_resampled.append(x_resampledi)
    y_resampled.append(y_resampledi)
#x_resampledj,y_resampledj = resample(x_reduced[75000:], y_train[75000:],num_seq)
#x_resampled.append(x_resampledj)
#y_resampled.append(y_resampledj)
x_resampled = np.array(np.concatenate(x_resampled, axis = 0))    
y_resampled = np.array(np.concatenate(y_resampled, axis = 0))
from random import shuffle
ind_list = [i for i in range(x_resampled.shape[0])]
shuffle(ind_list)
x_train_ = x_resampled[ind_list]
y_train_ = y_resampled[ind_list]

In [10]:
y_train_[:10]

array([0., 1., 0., 1., 1., 0., 0., 0., 1., 0.])

In [11]:
x_valid = x_train_[-40000:]
y_valid = y_train_[-40000:]
train_x = x_train_[:-40000]
train_y = y_train_[:-40000]
x_train.shape
np.mean(y_valid)

0.499125

In [12]:
x_train

array([[0.        , 0.5       , 0.28985507, ..., 0.        , 0.05      ,
        0.        ],
       [0.        , 1.        , 0.63768116, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.5       , 0.71014493, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.5       , 0.7826087 , ..., 0.        , 0.01      ,
        0.        ],
       [0.        , 0.5       , 0.43478261, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.5       , 0.28985507, ..., 0.        , 0.        ,
        0.        ]])

In [13]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(train_x,train_y)
val_pred = clf.predict(x_valid)
test_pred = clf.predict(x_test_reduced)

In [14]:
train_x.shape

(118862, 18)

In [15]:
from sklearn.metrics import confusion_matrix
tn,fp,fn,tp = confusion_matrix(y_valid,val_pred).ravel()
(tp,fp,fn,tp)

(18141, 89, 1824, 18141)

In [16]:
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.9951179374657159,
 0.004442226104317444,
 0.9086401202103681,
 0.949914910328577,
 0.952175)

In [17]:
tn,fp,fn,tp = confusion_matrix(y_test[:-1],test_pred).ravel()
(tp,fp,fn,tn)

(4596, 146, 1145, 16657)

In [18]:
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.9692113032475749,
 0.008688924596798191,
 0.8005573941821982,
 0.8768482304683773,
 0.9427342086586231)

In [19]:
train_pred = clf.predict(train_x)
tn,fp,fn,tp = confusion_matrix(train_y,train_pred).ravel()

acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.9963832299970475,
 0.0032998855141760388,
 0.9080146638415229,
 0.9501486916891024,
 0.9523312749238613)

In [20]:
def get_batches(arr,y, batch_size, num_seq=num_seq):
    new_arr = np.zeros((arr.size,num_seq))
    new_y = np.zeros(arr.size)
    for i in range(arr.size-num_seq):
        for j in range(num_seq):
            new_arr[i,j] = arr[i+j]
        new_y[i] = y[i+num_seq-1]
    n_batches = arr.shape[0]//batch_size
    for i in range(0,n_batches*batch_size,batch_size):
        x = new_arr[i:i+batch_size,:]
        y = new_y[i:i+batch_size]
        yield x,y
x, y = next(get_batches(train_pred, train_y,200))       
x.shape

def build_inputs(batch_size, num_inputs = 100 ):
    inputs = tf.placeholder(tf.float32, [None,num_inputs], name = 'inputs')
    outputs = tf.placeholder(tf.int32, [None,],name = 'targets')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, outputs, keep_prob
build_inputs(200)

def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Build LSTM cell.
    
        Arguments
        ---------
        keep_prob: Scalar tensor (tf.placeholder) for the dropout keep probability
        lstm_size: Size of the hidden layers in the LSTM cells
        num_layers: Number of LSTM layers
        batch_size: Batch size

    '''
    ### Build the LSTM Cell
    
    def build_cell(lstm_size, keep_prob):
        # Use a basic LSTM cell
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Add dropout to the cell
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Stack up multiple LSTM layers, for deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    initial_state = cell.zero_state(batch_size, tf.float32)
    return cell, initial_state   

def build_output(lstm_output, in_size, out_size, relu_size = 100):
    out = tf.contrib.layers.fully_connected(lstm_output, relu_size, activation_fn = tf.nn.relu)
    out = tf.layers.batch_normalization(out)
    logits = tf.contrib.layers.fully_connected(out,out_size, activation_fn=None)
    #logits = tf.reshape(logits, (-1,10,3, out_size))
    return logits


def build_loss(logits, targets,vocab_size):
    ''' Calculate the loss from the logits and the targets.
    
        Arguments
        ---------
        logits: Logits from final fully connected layer
        targets: Targets for supervised learning
        lstm_size: Number of LSTM hidden units
        num_classes: Number of classes in targets
        
    '''
    
      # One-hot encode targets and reshape to match logits, one row per batch_size per step
    #logits = tf.reshape(logits,[-1,vocab_size])
    y_one_hot = tf.one_hot(targets, vocab_size)
    #y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    #y_reshaped = tf.reduce_sum(y_one_hot, axis = 1)
    # Softmax cross entropy loss
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_one_hot)
    loss = tf.reduce_mean(loss)
    return loss, y_one_hot

def build_optimizer(loss, learning_rate, grad_clip):
    ''' Build optmizer for training, using gradient clipping.
    
        Arguments:
        loss: Network loss
        learning_rate: Learning rate for optimizer
    
    '''
    
    # Optimizer for training, using gradient clipping to control exploding gradients
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

class RNN:
    
    def __init__(self, vocab_size, relu_size, batch_size=64,
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5):
    
        # When we're using this network for sampling later, we'll be passing in
        # one character at a time, so providing an option for that

        tf.reset_default_graph()
        
        # Build the input placeholder tensors
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_inputs = num_seq)
        
        # Build the LSTM cell
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)
        
        ### Run the data through the RNN layers
        # First, one-hot encode the input tokens
       
        
        # Run each sequence step through the RNN and collect the outputs
        outputs, state = tf.nn.dynamic_rnn(cell, tf.reshape(self.inputs,[-1,num_seq,1]), initial_state=self.initial_state)
        self.final_state = state
        
        # Get softmax predictions and logits
        self.logits = build_output(outputs, lstm_size, vocab_size, relu_size = relu_size)[:,-1]
        
        # Loss and optimizer (with gradient clipping)
        self.loss, self.y_reshaped = build_loss(self.logits, self.targets, vocab_size)
        self.pred = tf.argmax(self.logits,1)
        #_,self.pred = tf.nn.top_k(self.logits,3)
        correct_pred = tf.equal(self.pred, tf.argmax(self.y_reshaped,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

save_every_n = 200

model = RNN(2,relu_size, batch_size=batch_size,
                lstm_size=rnn_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/______.ckpt')
    counter = 0
    for e in range(epochs):
        train_loss = []
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for x, y in get_batches(train_pred, train_y,batch_size):
            
            counter += 1
            
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: keep_prob,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.loss, 
                                                 model.final_state, 
                                                 model.optimizer], 
                                                 feed_dict=feed)
            train_loss.append(batch_loss)
            #if(counter%show_every_n_batches ==0):
                #print('Epoch: {}/{}... '.format(e+1, epochs),
                  #'Training Step: {}... '.format(counter),
                    #'Training loss: {:.4f}... '.format(batch_loss))
        
            if (counter % save_every_n == 0):
                saver.save(sess, "checkpoints/i{}_l{}.ckpt".format(counter, num_layers))
        val_loss = []
        val_state = sess.run(model.initial_state)
        for x, y in get_batches(val_pred, y_valid, batch_size):
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
            batch_loss, val_state = sess.run([model.loss, 
                                                 model.final_state], 
                                                 feed_dict=feed)
            
                                                 
            val_loss.append(batch_loss)
            
        print("epoch {}: Training loss : {:.6f}".format(e, np.mean(train_loss)))
        print("Validation loss : {:.6f}".format(np.mean(val_loss)))
        
    val_acc = []
    valid_pred =[]
    val_y = []
    val_logits = []
    val_state = sess.run(model.initial_state)
    for x, y in get_batches(val_pred, y_valid,batch_size):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
        val_state,batch_pred,batch_acc,batch_logits = sess.run([model.final_state,model.pred,model.accuracy, model.logits], feed_dict = feed)
        val_acc.append(batch_acc)
        val_y.append(y)
        valid_pred.append(batch_pred)
        val_logits.append(batch_logits)
    training_pred =[]
    train_label = []
    train_acc = []
    train_logits = []
    train_state = sess.run(model.initial_state)
    for x, y in get_batches(train_pred, train_y,batch_size):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: train_state}
        train_state,batch_pred,batch_acc, batch_logits = sess.run([model.final_state,model.pred,model.accuracy, model.logits], feed_dict = feed)
        train_acc.append(batch_acc)
        train_label.append(y)
        training_pred.append(batch_pred)
        train_logits.append(batch_logits)
    print("Training accuracy : {:.2f}".format(100*np.mean(train_acc)))    
    print("validation accuracy : {:.2f}".format(100*np.mean(val_acc)))
    test_acc = []
    test_pred_ =[]
    test_y = []
    test_logits = []
    test_state = sess.run(model.initial_state)
    for x, y in get_batches(test_pred, y_test,batch_size,num_seq):
        feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 1,
                    model.initial_state: val_state}
        test_state,batch_pred,batch_acc, batch_logits = sess.run([model.final_state,model.pred,model.accuracy,model.logits], feed_dict = feed)
        test_acc.append(batch_acc)
        test_y.append(y)
        test_pred_.append(batch_pred)
        test_logits.append(batch_logits)
    print("Test accuracy : {:.2f}".format(100*np.mean(test_acc))) 

tn,fp,fn,tp = confusion_matrix(np.concatenate(test_y),np.concatenate(test_pred_)).ravel()
(tp,fp,fn,tn)
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

epoch 0: Training loss : 0.551019
Validation loss : 0.332696
epoch 1: Training loss : 0.225657
Validation loss : 0.181597
epoch 2: Training loss : 0.174013
Validation loss : 0.171841
epoch 3: Training loss : 0.169996
Validation loss : 0.170896
epoch 4: Training loss : 0.169372
Validation loss : 0.170925
epoch 5: Training loss : 0.169221
Validation loss : 0.170944
epoch 6: Training loss : 0.169164
Validation loss : 0.170983
epoch 7: Training loss : 0.169136
Validation loss : 0.171004
epoch 8: Training loss : 0.169121
Validation loss : 0.171015
epoch 9: Training loss : 0.169111
Validation loss : 0.171028
epoch 10: Training loss : 0.169096
Validation loss : 0.171039
epoch 11: Training loss : 0.169086
Validation loss : 0.171048
epoch 12: Training loss : 0.169082
Validation loss : 0.171056
epoch 13: Train

(0.9692340335242945,
 0.00868471490177288,
 0.8008415147265077,
 0.8770279351060766,
 0.9428125)

In [21]:

tn,fp,fn,tp = confusion_matrix(np.concatenate(val_y),np.concatenate(valid_pred)).ravel()
(tp,fp,fn,tn)
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.9951050489495105,
 0.0044309469282087025,
 0.908556794215125,
 0.9498635027299455,
 0.95225)

In [22]:
tn,fp,fn,tp = confusion_matrix(np.concatenate(train_label),np.concatenate(training_pred)).ravel()
(tp,fp,fn,tn)
acc = (tp+tn)/(tp+tn+fp+fn)
DR = tp/(tp+fp)
FAR = fp/(tn+fp)
recall = tp/(tp+fn)
f1 = 2*(recall*DR)/(recall+DR)
(DR,FAR,recall,f1,acc)

(0.9963802910541479,
 0.003300663500724125,
 0.9080076744420883,
 0.9501435288730783,
 0.9523400673400674)